# Optimising simulations for a generic loss function

This notebook showcases optimising a design for some generic loss function $\mathcal{L}$ using an event loop calling **Elmer** or **HFSS**.
As such, this notebook should be run on a machine with the simulation software, KLayout and necessary Python packages installed.
This is similar to Ansys Optimetrics but more general due to supporting arbitrary KQCircuits geometry,
Elmer and, any search algorithms and schedulers.
The current implementation requires installing [`ray`](https://docs.ray.io/en/releases-2.0.0/index.html)
```bash
pip install "ray[tune]"==2.1.0
```
and the optimisers generally demand additional manual installation. For example, this notebook employs [`Optuna`](https://optuna.readthedocs.io/en/stable/) and we need to also install:
```bash
pip install optuna==3.0.4 botorch==0.7.3 scikit-learn==1.1.3 plotly==5.11.0
```
Newer versions may work but these are the ones the notebook was written with.

## Workflow

1. Supply your ``Simulation`` class, `sim_parameters`, and `export_parameters` as usual in the [Simulation settings](#simulation-settings)
2. Implement fetching simulation results for your use case and a meaningful loss function $\mathcal{L}$ in the [Loss settings](#loss-settings) section.
3. Specify the variables that can be varied by the optimiser and their ranges in [Optimisation settings](#optimisation-settings). See [Ray Tune - Random Distributions API](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/search_space.html?highlight=quniform#random-distributions-api) for possible (and custom) distributions.
4. Select your optimiser and its settings. You can also try using a simpler framework, such as, `scipy`, `pytorch`, `tensorflow` etc. See [Ray Tune - Search Algorithms](https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html) for algorithms with support out-of-the-box.
5. Run the optimiser (`ray` tuning)
6. View the results! You can manipulate the DataFrame, or use [Tensorboard](https://www.tensorflow.org/tensorboard) to view the data in the same folder as this notebook:
   ```bash
   pip install tensorboard
   tensorboard --logdir=./ray_results
   ```


The _toy example_ implemented in this notebook is optimising the qubit $C_\Sigma$ ``DoublePads`` to be $65\,\text{fF}$.

In [1]:
import os
import re
import json
import shutil
import subprocess
import importlib
from pathlib import Path
from functools import partial
from datetime import datetime

import pandas as pd
import ray
import ray.air
import ray.air.session
from ray import tune

from kqcircuits.pya_resolver import pya
from kqcircuits.defaults import STARTUPINFO
from kqcircuits.util.geometry_json_encoder import GeometryJsonEncoder
from kqcircuits.simulations.export.ansys.ansys_export import export_ansys
from kqcircuits.simulations.export.elmer.elmer_export import export_elmer
from kqcircuits.simulations.export.simulation_export import export_simulation_oas
from kqcircuits.util.export_helper import create_or_empty_tmp_directory, get_active_or_new_layout

if os.environ.get('KQC_TMP_PATH') is None:
    os.environ['KQC_TMP_PATH'] = os.getcwd()
original_kqc_tmp_path = os.environ['KQC_TMP_PATH']

### Please set the location of your interpreter for `.sh` files

Some examples are given below

In [2]:
# Please set the location of your interpreter for `.sh` files, some examples are given below.
# The interpreter will be called with `-c SCRIPT.sh`.
# If this does not work, try adjusting the `subprocess.run` call later in the notebook
sh_interpreter = 'sh'
sh_interpreter = shutil.which('bash') if os.name != 'nt' else 'wsl -e bash'  # use WSL on Windows
sh_interpreter = r'C:\Program Files\Git\bin\bash.exe'

if shutil.which(sh_interpreter) is None:
    raise FileNotFoundError(f"Your shell `{sh_interpreter=}` is not found! Please set it to a working shell.")

tool = ['ansys', 'elmer'][1]
# This needs to be only as high as the number of licenses when using Ansys
n_workers = 5 if tool == 'elmer' else 1  # first-level parellelisation
n_processes = 4 if tool == 'elmer' else 5  # second-level parellelisation

## Implement helper functions

In [3]:
def create_and_export_sim(params, sim_class, sim_parameters, export_parameters, i):
    """Creates and exports a :class:`.Simulation` with given parameters.

    Arguments:
        params: Iteration-specific parameters to use. Combined with ``sim_parameters``.
        sim_class (Simulation): Given simulation. Should be loadable from a module (not anonymous).
        sim_parameters (dict): Dict of nominal simulation parameters.
        export_parameters (dict): Dict of simulation export parameters. May be for Ansys or Elmer
        i (str): Unique identifier for this iteration.

    Returns:
        dir_path: Path to exported simulations folder.
    """

    # Find corresponding class, has to be done this way to avoid pickling
    module = importlib.import_module(sim_class[0])
    sim_class = getattr(module, sim_class[1])

    dir_path = create_or_empty_tmp_directory(sim_parameters['name'] + f"_opt_{i}")
    export_parameters = {'path': dir_path, **export_parameters}

    # Get layout
    layout = get_active_or_new_layout()

    # Nominal simulation
    simulations = [sim_class(layout, **{
        **sim_parameters,
        **params
    })]

    if 'ansys_tool' in export_parameters:
        export_ansys(simulations, **export_parameters)
    else:
        export_elmer(simulations, **export_parameters)

    # Write oas files
    export_simulation_oas(simulations, dir_path)

    return dir_path

Implement small serializer for DBoxes, as it is often needed in `sim_parameters`.

In [4]:
def dbox_serializer(dbox):
    return GeometryJsonEncoder.encode_geometry(dbox)

def dbox_deserializer(o):
    return pya.DBox(
        pya.DPoint(o[0][0], o[0][1]), pya.DPoint(o[1][1], o[1][1])
    )

ray.util.register_serializer(pya.DBox, serializer=dbox_serializer, deserializer=dbox_deserializer)

## Simulation settings

Specify simulation export settings as normal

In [5]:
# pylint: disable=invalid-name,wrong-import-position
from kqcircuits.simulations.double_pads import DoublePadsSim

sim_class = DoublePadsSim

# Nominal simulation parameters
sim_parameters = {
    'name': re.sub(r'(?<!^)(?=[A-Z])', '_', sim_class.__name__).lower(),  # convert to snake_case
    'use_internal_ports': True,
    'use_ports': True,
    'internal_island_ports': True,
    'box': pya.DBox(pya.DPoint(0,0), pya.DPoint(2000, 2000)),
    'wafer_stack_type': 'planar'
}

#########
# Ansys #
#########

export_parameters_ansys_q3d = {
    'ansys_tool': 'q3d',
    'exit_after_run': True,
    'percent_error': 0.3,
    'minimum_converged_passes': 2,
    'maximum_passes': 20,
}

export_parameters_ansys_epr = {
    'ansys_tool': 'eigenmode',
    'exit_after_run': True,
    'max_delta_f': 0.5,

    # do two passes with tight mesh
    'gap_max_element_length': 20,
    'maximum_passes': 2,
    'minimum_passes': 1,
    'minimum_converged_passes': 1,

    # lossy eigenmode simulation settings
    'n_modes': 1,
    'frequency': 0.5,  # minimum allowed eigenfrequency
    'simulation_flags': ['pyepr'],
    'substrate_loss_tangent': 1e-6,

    # run T1 analysis with pyEPR between simulations
    'intermediate_processing_command': 'python "scripts/t1_estimate.py"',
    'participation_sheet_distance': 5e-3,  # in µm
    'dielectric_surfaces': {
        'layerMA': {
            'tan_delta_surf': 0.001,  # loss tangent
            'th': 4.8e-9,  # thickness
            'eps_r': 10,  # relative permittivity
        },
        'layerMS': {
            'tan_delta_surf': 0.001,
            'th': 0.3e-9,  # estimate worst case
            'eps_r': 10,
        },
        'layerSA': {
            'tan_delta_surf': 0.001,
            'th': 2.4e-9,
            'eps_r': 10,
        }
    },
}

#########
# Elmer #
#########

export_parameters_elmer = {
    'tool': 'capacitance',
    'workflow': {
        'python_executable': 'python',  # use 'kqclib' when using singularity
        'run_gmsh_gui': False,
        'elmer_n_processes': n_processes,  # -1 means all the physical cores
        'gmsh_n_threads': n_processes,
    },
    'gmsh_params': {
        'default_mesh_size': 100.,
        'gap_min_mesh_size': 2.,
        'gap_min_dist': 4.,
        'gap_max_dist': 200.,
        'port_min_mesh_size': 1.,
        'port_min_dist': 4.,
        'port_max_dist': 200.,
        'algorithm': 5,
    }
}

We choose to use Elmer for the capacitance simulations

In [6]:
export_parameters = export_parameters_elmer  # use Elmer
# export_parameters = export_parameters_ansys_epr  # use Ansys + pyEPR

## Loss settings

These need to be adjusted to fit your specific use case.
The optimiser is set to minimise. If you want to maximise, you can modify $\mathcal{L} \to -\mathcal{L}$.
Furthermore, the simplest way to attain a specific value is to set the loss function to
$$
\mathcal{L}(x) = \left| x_\text{desired} - x \right|
$$
This is not differentiable at $\mathcal{L}(x_\text{desired})$, but appears to work decently.
A common alternative is to use the mean squared error ($L_2$-norm).

In [7]:
def load_results_Q(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('Qdata*.csv'))[0]
    df = pd.read_csv(result_file)
    return df.iloc[0].to_dict()


def loss_Q(data):
    r"""Loss function :math:`$\mathcal(L)` to minimum loss :math:`\delta`."""
    Q = data['Q_total']
    return 1 / Q, Q


def load_results_capacitance(dir_path):
    """Function to load simulation results. Input is simulation folder path."""
    result_file = list(Path(dir_path).rglob('*_project_results.json'))[0]
    with open(result_file) as fp:
        data = json.load(fp)
    return data


def loss_C_Sigma_two_islands(data, C):
    r""" Data argument is a dict with `CMatrix` key of a 3x3 capacitance matrix with a coupler as the last port.

    Returns:
        loss: MSE loss to target ``C``
        C_Sigma: absolute value for :math:`C_\Sigma`
    """
    def _reciprocal_sum(*args):
        return 1 / sum((1 / e for e in args))

    C_sim = data['CMatrix']
    C_first_island = C_sim[0][0] + _reciprocal_sum(C_sim[0][2], C_sim[2][2])
    C_second_island = C_sim[1][1] + _reciprocal_sum(C_sim[1][2], C_sim[2][2])
    C_sigma = C_sim[0][1] + _reciprocal_sum(C_first_island, C_second_island)
    return ((C - C_sigma) * 1e15) ** 2, C_sigma


def loss_capacitance(data, C, i=0, j=0):
    r"""Loss function :math:`$\mathcal(L)` to get certain capacitance.

    Arguments:
        data: simulation results dictionary containing ``CMatrix``
        C: desired capacitance in farads
        i: first index for the desired capacitance from the ``CMatrix``
        j: second index for the desired capacitance from the ``CMatrix``

    Returns:
        loss: The computed loss
        x: Value with which the loss was computed

    Usage:

        .. code-block:: python

            from functools import partial
            loss = partial(lossC, C=50e-15, i=1, j=0)
            # Do the following inside the event loop
            results = load_results(dir_path)
            score = loss(results)
    """
    C_sim = data['CMatrix'][i][j]
    return ((C - C_sim) * 1e15) ** 2, C_sim  # Optimiser might get confused by small numbers

Select your loss function to use

In [8]:
load_results = load_results_capacitance

# loss = partial(loss_capacitance, C=100e-15, i=0, j=1)
loss = partial(loss_C_Sigma_two_islands, C=65e-15)

# load_results = load_results_Q
# loss = loss_Q

## Optimisation settings

Specify parameter search spaces, optimiser (such as ``HyperOptSearch``), possible scheduler, maximum iterations,
and parallel workers (only with Elmer).

In [9]:
search_config = {
    'params': {
        'island1_extent_[0]': tune.uniform(200, 700),
        'island1_extent_[1]': tune.uniform(50, 500),

        'island2_extent_[0]': tune.uniform(200, 700),
        'island2_extent_[1]': tune.uniform(50, 300),

        'coupler_extent[0]': tune.uniform(40, 500),
        'coupler_extent[1]': tune.uniform(20, 40),
        'coupler_offset': tune.uniform(5, 50),

        'ground_gap_[0]': tune.uniform(550, 1000),
        'ground_gap_[1]': tune.uniform(400, 1000),

        'squid_offset': tune.uniform(-20, 0),
        'island1_r': tune.uniform(0, 50),
        'island2_r': tune.uniform(0, 50),
    },
}

def constraints(trial):
    """ Return Sequence of (soft) constraints for the optimiser. Positive values are punished and <= 0 are ok.
    See optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.BoTorchSampler.html for details.
    """
    c0 = trial.params['params/ground_gap'] + trial.params['params/r_outer'] - 150
    c1 = trial.params['params/r_inner'] - trial.params['params/r_outer']  # negative is ok
    return [c0, c1]  # tries to optimise equal to 0

# Avoid more pickling of KLayout classes and don't show these as search params
create_and_export = partial(
    create_and_export_sim,
    sim_parameters=sim_parameters,
    export_parameters=export_parameters,
    sim_class=(sim_class.__module__, sim_class.__name__),  # we avoid pickling the class by loading this way
)

# Import chosen optimiser and scheduler. NB: Selecting a new optimiser often requires you to install a separate package
# See https://docs.ray.io/en/releases-2.0.0/tune/api_docs/suggestion.html

# pylint: disable=wrong-import-position,ungrouped-imports
from ray.tune.search.optuna import OptunaSearch
import optuna
import joblib  # shold come with Optuna


tune_config = tune.TuneConfig(
    metric='score',
    mode='min',
    search_alg=(optuna_search := OptunaSearch(
        # If you want soft constraints
        sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),
        # sampler=optuna.integration.BoTorchSampler(),
        metric=['score'],  #, 'outer_island_width'],
        mode=['min']  #, 'min'],
    )),
    ## Examples of other possible optimisers (search algorihtms)
    # search_alg=BayesOptSearch(),
    # search_alg=HEBOSearch(),
    # search_alg=DragonflySearch(
    #     metric='score',
    #     mode='min',
    #     optimizer='bandit',
    #     domain='Euclidean'
    # ),
    max_concurrent_trials=n_workers,  # Has to be 1 for Ansys, may be more for Elmer
    num_samples=50,  # max iterations, can be -1 for infinite
    time_budget_s=0,  # Time after which optimisation is stopped. May be useful along with ``num_samples=-1``.
)

C:\Users\NikoSavola\AppData\Local\Temp\ipykernel_11532\104588506.py:52: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler=optuna.integration.BoTorchSampler(constraints_func=constraints),


In [10]:
def event_loop(config):

    dir_path = create_and_export(**config, i=ray.air.session.get_trial_id())
    use_ansys = 'ansys_tool' in create_and_export.keywords['export_parameters']

    # Combine parameters with the following format to single array param:
    #   param_[0] + param_[1] -> param = [param_[0], param_[1]]
    arr_match = r'_\[(\d)\]'
    config['params'] |= {
        (arr_key := re.sub(arr_match, '', key)): config['params'].get(arr_key, []) + [config['params'][key]]
        for key in config['params']
        if re.match(arr_match, key)
    }

    subprocess.run(
        ['simulation.bat'] if use_ansys else [sh_interpreter, '-c', './simulation.sh'],
        cwd=dir_path,
        shell=True,
        check=True,
        startupinfo=STARTUPINFO
    )

    results = load_results(dir_path)
    score, x = loss(results)
    return {'score': score, 'value': x}
    # For multi-objective, just return the different loss functions and specify in `metric`, e.g.
    # from math import prod
    # return {'score': score, 'value': x, 'island1_size': math.prod(config['params']['island1_extent'])}

## Run optimisation

Start a Tune run and print the results.

In [11]:
# Set folder for optimisation files
os.environ['KQC_TMP_PATH'] = (
    Path(original_kqc_tmp_path) / (sim_class.__name__ + '_' + str(datetime.now()))\
        .replace(' ', '_').replace(':', '.')
).as_posix()

tuner = tune.Tuner(
    tune.with_resources(event_loop, {"cpu": n_processes}),  # maximum resources given to worker
    param_space=search_config,
    tune_config=tune_config,
    run_config=ray.air.RunConfig(
        local_dir=os.environ['KQC_TMP_PATH'] + "/ray_results",  # needed for tensorboard
        checkpoint_config=ray.air.CheckpointConfig(
            checkpoint_frequency=1
        ),
        log_to_file=True
    )
)

# For resuming a run
# tuner = tune.Tuner.restore(os.environ['KQC_TMP_PATH'] + "/ray_results" + "/some_folder", restart_errored=False)

results = tuner.fit()

# This is specific only for the Optuna optimiser, we also save the 'study' object
study_file = Path(results._experiment_analysis.runner_data()['_local_checkpoint_dir']) / "study.pkl"
if study_file.exists():
    print(f'Loading study from {study_file}')
    study = joblib.load(study_file)
else:
    print(f'Saving study to {study_file}')
    study = optuna_search._ot_study
    joblib.dump(study, study_file)

os.environ['KQC_TMP_PATH'] = original_kqc_tmp_path

2022-12-13 11:53:21,606	INFO worker.py:1528 -- Started a local Ray instance.
2022-12-13 11:53:23,280	WARNING function_trainable.py:586 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\ray\tune\search\optuna\optuna_search.py:682: FutureWarning: UniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.UniformDistribution(
[I 2022-12-13 11:53:23,285] A new study created in memory with name: optuna


C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=700.0, low=200.0) is deprecated and internally converted to FloatDistribution(high=700.0, log=False, low=200.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=500.0, low=50.0) is deprecated and internally converted to FloatDistribution(high=500.0, log=False, low=50.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\distributions.py:766: FutureWarning: UniformDistribution(high=300.0, low=50.0) is deprecated and internally converted to FloatDistribution(high=300.0, log=False, low=50.0, step=None). See https://github.com/optuna/optuna/issues/294

(event_loop pid=17340) Submitting the main script of simulation 1/1
(event_loop pid=17340) --------------------------------------------
(event_loop pid=17340) Simulation 1/1 Gmsh


(event_loop pid=13016) Ground location of port 1 is not determined.
(event_loop pid=13016) Ground location of port 2 is not determined.
(event_loop pid=35760) Ground location of port 1 is not determined.
(event_loop pid=35760) Ground location of port 2 is not determined.
(event_loop pid=37848) Ground location of port 1 is not determined.
(event_loop pid=37848) Ground location of port 2 is not determined.
(event_loop pid=18252) Ground location of port 1 is not determined.
(event_loop pid=18252) Ground location of port 2 is not determined.


(event_loop pid=13016) Submitting the main script of simulation 1/1
(event_loop pid=13016) --------------------------------------------
(event_loop pid=13016) Simulation 1/1 Gmsh
(event_loop pid=37848) Submitting the main script of simulation 1/1
(event_loop pid=37848) --------------------------------------------
(event_loop pid=35760) Submitting the main script of simulation 1/1
(event_loop pid=35760) --------------------------------------------
(event_loop pid=35760) Simulation 1/1 Gmsh
(event_loop pid=37848) Simulation 1/1 Gmsh
(event_loop pid=18252) Submitting the main script of simulation 1/1
(event_loop pid=18252) --------------------------------------------
(event_loop pid=18252) Simulation 1/1 Gmsh
(event_loop pid=17340) Simulation 1/1 ElmerGrid
(event_loop pid=37848) Simulation 1/1 ElmerGrid
(event_loop pid=35760) Simulation 1/1 ElmerGrid
(event_loop pid=13016) Simulation 1/1 ElmerGrid
(event_loop pid=18252) Simulation 1/1 ElmerGrid
(event_loop pid=17340) Simulation 1/1 Elmer


...

(event_loop pid=32316) Simulation 1/1 write results json
(event_loop pid=41148) Simulation 1/1 Paraview


2022-12-13 11:58:00,115	WARNING optuna_search.py:549 -- 'params/ground_gap'


(event_loop pid=41148) Simulation 1/1 write results json


2022-12-13 11:58:00,233	WARNING optuna_search.py:549 -- 'params/ground_gap'
2022-12-13 11:58:00,897	WARNING optuna_search.py:549 -- 'params/ground_gap'
2022-12-13 11:58:01,043	INFO tune.py:777 -- Total run time: 277.76 seconds (277.61 seconds for the tuning loop).


Saving study to D:\LocalSimulations\DoublePadsSim_2022-12-13_11.53.18.886041\ray_results\event_loop_2022-12-13_11-53-18\study.pkl


This is specific only for the Optuna optimiser

In [12]:
# Default 'importance' metric is fANOVA, see optuna.importance.FanovaImportanceEvaluator
fig = optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0])
fig.show()

fig = optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[0])
fig.show()

try:
    fig = optuna.visualization.plot_pareto_front(study)  # or optuna.visualization.matplotlib.plot_pareto_front(study)
    fig.show()
except ValueError as e:
    print('Not multi-objective.', e)

C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:69: UserWarning: `target` is specified, but `target_name` is the default value, 'Objective Value'.
  warnings.warn(


C:\Users\NikoSavola\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_utils.py:69: UserWarning:

`target` is specified, but `target_name` is the default value, 'Objective Value'.



Not multi-objective. `plot_pareto_front` function only supports 2 or 3 objective studies when using `targets` is `None`. Please use `targets` if your objective studies have more than 3 objectives.


In [13]:
df = results.get_dataframe()
best = results.get_best_result(metric="score", mode="min")

display(df.filter(regex='(score|value|time_this|config)'))
print(f"{best.metrics['value']=}", best.config)

,score,value,time_this_iter_s,config/params/coupler_extent[0],config/params/coupler_extent[1],config/params/coupler_offset,config/params/ground_gap_[0],config/params/ground_gap_[1],config/params/island1_extent_[0],config/params/island1_extent_[1],config/params/island1_r,config/params/island2_extent_[0],config/params/island2_extent_[1],config/params/island2_r,config/params/squid_offset
0,2.549566e+00,6.659674e-14,22.942004,184.819356,32.892083,11.876995,902.545600,885.957344,657.323140,78.531658,0.386683,505.288865,59.834871,31.540880,-12.686303
1,2.856388e-01,6.553445e-14,23.764786,262.164333,21.901356,10.025965,941.073518,872.945683,557.517771,55.547059,33.638677,406.106803,172.876456,38.146902,-3.406442
2,2.321564e+00,6.652367e-14,23.097740,390.579356,39.496010,9.312510,707.919401,744.302212,330.271422,145.703800,0.090669,319.115045,216.700491,34.597832,-1.908861
3,3.080977e+00,6.675527e-14,22.944740,252.262631,35.014137,19.579834,967.728027,601.337263,576.230569,194.289542,8.926458,582.846310,123.523916,11.835129,-18.334183
4,1.140939e+00,6.393185e-14,23.748721,95.950940,28.118845,23.925636,809.682592,971.087702,369.147366,360.972920,48.528605,204.677578,197.706363,49.861041,-5.338458
5,8.851016e-02,6.529751e-14,25.912620,358.622417,32.541733,40.691043,720.716543,741.044599,695.014301,271.086154,28.186896,641.766179,162.848684,27.705481,-16.325667
6,2.247669e-01,6.547410e-14,25.893102,441.755062,30.086347,18.458191,615.763386,587.442759,492.917836,125.800562,10.445583,479.983364,195.970535,45.154686,-4.347621
7,1.816981e-01,6.542626e-14,25.879092,145.852582,37.039412,15.130161,946.686004,671.214726,226.104242,424.334801,32.795276,462.665960,276.240548,35.266262,-12.009317
8,3.434012e-01,6.558600e-14,24.943299,115.966432,29.117388,32.317955,984.626256,687.294723,280.298611,354.075378,7.416354,261.138651,269.898192,36.551891,-2.089273
9,1.864357e+00,6.636541e-14,22.635954,280.739250,38.104601,33.694265,640.324633,540.970411,691.877462,352.067370,25.795012,332.112030,138.432974,6.226970,-4.015939


best.metrics['value']=6.499929719839244e-14 {'params': {'island1_extent_[0]': 606.6768661584701, 'island1_extent_[1]': 310.75900926854484, 'island2_extent_[0]': 655.1719453338441, 'island2_extent_[1]': 141.44193907885966, 'coupler_extent[0]': 272.69555554348483, 'coupler_extent[1]': 27.89008906104139, 'coupler_offset': 40.97286230864692, 'ground_gap_[0]': 635.2675550392195, 'ground_gap_[1]': 814.9529213133264, 'squid_offset': -9.667677878700868, 'island1_r': 23.942272036879324, 'island2_r': 40.91836630120784}}
